## Sparse Optical Flow

In [1]:
import cv2
import numpy as np

In [31]:
# Open the video file
cap = cv2.VideoCapture('railway.webm')

In [32]:
ret, frame1 = cap.read()
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Detect keypoints in the first frame
feature_params = dict(maxCorners=500,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

# Create a mask for drawing purposes
mask = np.zeros_like(frame1)

In [33]:
while True:
    # Read the current frame
    ret, frame2 = cap.read()
    if not ret:
        break

    # Convert the current frame to grayscale
    gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate sparse optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel().astype(int)
        c, d = old.ravel().astype(int)
        mask = cv2.line(mask, (a, b), (c, d), (0, 255, 0), 2)
        frame2 = cv2.circle(frame2, (a, b), 5, (0, 255, 0), -1)

    img = cv2.add(frame2, mask)

    # Display the output frame
    cv2.imshow('Sparse Optical Flow', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Update previous frame and points
    prev_gray = gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()